# Demographic data by geography

In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
import glob
import math
import os
import re

import pandas as pd

# Los Angeles County Continuum of Care

Los Angeles County comprises four CoCs. Thanks to [this crosswalk data](https://github.com/tomhbyrne/HUD-CoC-Geography-Crosswalk), I'm able to use demographic data for the subset of the county that exists within CoC ID CA-600.

## Normalize

These are slightly different for each of the ACS 5-Year datasets I plan to use. Based on previous experience, column IDs can also vary among years, so I'm going the inelegant route because the names are more immediately legible to me.

In [3]:
usecols_2019 = [
    'id',
    'Geographic Area Name',
    'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino',
    'Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino',
    'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
    'Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
    'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!White alone',
    'Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!White alone',
    'Estimate!!RACE!!Total population',
    'Margin of Error!!RACE!!Total population',
    'Estimate!!RACE!!Total population!!One race!!White',
    'Margin of Error!!RACE!!Total population!!One race!!White',
    'Estimate!!RACE!!Total population!!One race!!Black or African American',
    'Margin of Error!!RACE!!Total population!!One race!!Black or African American',
    'Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native',
    'Margin of Error!!RACE!!Total population!!One race!!American Indian and Alaska Native',
    'Estimate!!RACE!!Total population!!One race!!Asian',
    'Margin of Error!!RACE!!Total population!!One race!!Asian',
    'Estimate!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander',
    'Margin of Error!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander',
]

In [4]:
def read_la_dp05_2019(file):
    year = os.path.basename(file)[7:11]
    df = pd.read_csv(file, skiprows=1, usecols=usecols_2019)
    df.columns = [
        re.sub('RACE!!', '', col)
        for col in df.columns
    ]
    df.columns = [
        re.sub('One race!!', '', col)
        for col in df.columns
    ]
    df.columns = [
        re.sub('Total population!!', '', col)
        for col in df.columns
    ]
    df.columns = [
        re.sub('HISPANIC OR LATINO AND ', '', col)
        for col in df.columns
    ]
    df.columns = [
        re.sub('Estimate Margin', 'Margin', col)
        for col in df.columns
    ]
    df['census year'] = year
    return df

In [5]:
la_dp05_2019 = read_la_dp05_2019(
    '../01_inputs/USCB/DP05/LA_2019/ACSDP5Y2019.DP05_data_with_overlays_2021-12-02T194643.csv')

In [6]:
la_dp05_2019.head(2)

,Estimate!!Total population,Margin of Error!!Total population,Estimate!!White,Margin of Error!!White,Estimate!!Black or African American,Margin of Error!!Black or African American,Estimate!!American Indian and Alaska Native,Margin of Error!!American Indian and Alaska Native,Estimate!!Asian,Margin of Error!!Asian,...,Margin of Error!!Native Hawaiian and Other Pacific Islander,Estimate!!Hispanic or Latino (of any race),Margin of Error!!Hispanic or Latino (of any race),Estimate!!Not Hispanic or Latino,Margin of Error!!Not Hispanic or Latino,Estimate!!Not Hispanic or Latino!!White alone,Margin of Error!!Not Hispanic or Latino!!White alone,id,Geographic Area Name,census year
0,4283,443,3347,353,19,15,3,6,331,142,...,6,1188,414,3095,354,2616,306,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali...",2019
1,3405,334,2981,343,74,102,15,24,220,156,...,12,147,86,3258,344,2882,358,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali...",2019


In [7]:
reshaped_la_dp05_2019 = la_dp05_2019.melt(
    id_vars=['census year', 'id', 'Geographic Area Name'])

In [8]:
reshaped_la_dp05_2019.head(2)

,census year,id,Geographic Area Name,variable,value
0,2019,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali...",Estimate!!Total population,4283
1,2019,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali...",Estimate!!Total population,3405


In [9]:
reshaped_la_dp05_2019[['measure', 'census demographic']] = reshaped_la_dp05_2019['variable'].str.split(
    '!!', 1, expand=True)

In [10]:
reshaped_la_dp05_2019['census demographic'].unique()

array(['Total population', 'White', 'Black or African American',
       'American Indian and Alaska Native', 'Asian',
       'Native Hawaiian and Other Pacific Islander',
       'Hispanic or Latino (of any race)', 'Not Hispanic or Latino',
       'Not Hispanic or Latino!!White alone'], dtype=object)

In [11]:
reshaped_la_dp05_2019['census demographic'] = reshaped_la_dp05_2019['census demographic'].replace(
    {'Not Hispanic or Latino!!White alone': 'White alone'})

In [12]:
reshaped_la_dp05_2019['census demographic'].unique()

array(['Total population', 'White', 'Black or African American',
       'American Indian and Alaska Native', 'Asian',
       'Native Hawaiian and Other Pacific Islander',
       'Hispanic or Latino (of any race)', 'Not Hispanic or Latino',
       'White alone'], dtype=object)

## Tract/CoC Crosswalk

[source](https://github.com/tomhbyrne/HUD-CoC-Geography-Crosswalk)

In [13]:
tract_crosswalk_df = pd.read_csv(
    '../01_inputs/USCB/tract_coc_match.csv',
    encoding='ISO-8859-1',
    dtype={'tract_fips': str},
)

In [14]:
la_tracts = tract_crosswalk_df[tract_crosswalk_df['coc_number']
                               == 'CA-600'].copy()

In [15]:
la_tracts['tract_fips'] = '1400000US' + la_tracts['tract_fips']

In [16]:
la_coc_tracts = set(la_tracts['tract_fips'])

In [17]:
la_coc_df = reshaped_la_dp05_2019[(reshaped_la_dp05_2019['id'].isin(la_coc_tracts)) & (
    reshaped_la_dp05_2019['census demographic'] != 'Total population')].copy()

In [18]:
la_coc_df.head(2)

,census year,id,Geographic Area Name,variable,value,measure,census demographic
4692,2019,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali...",Estimate!!White,3347,Estimate,White
4693,2019,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali...",Estimate!!White,2981,Estimate,White


In [19]:
la_coc_total_df = reshaped_la_dp05_2019[(reshaped_la_dp05_2019['id'].isin(la_coc_tracts)) & (
    reshaped_la_dp05_2019['census demographic'] == 'Total population')].copy()

Totals to be used in percent calculation [below](#Combine-LA-CoC-and-county-data)

In [20]:
la_coc_total_df

,census year,id,Geographic Area Name,variable,value,measure,census demographic
0,2019,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali...",Estimate!!Total population,4283,Estimate,Total population
1,2019,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali...",Estimate!!Total population,3405,Estimate,Total population
2,2019,1400000US06037101210,"Census Tract 1012.10, Los Angeles County, Cali...",Estimate!!Total population,6347,Estimate,Total population
3,2019,1400000US06037101220,"Census Tract 1012.20, Los Angeles County, Cali...",Estimate!!Total population,3702,Estimate,Total population
4,2019,1400000US06037101300,"Census Tract 1013, Los Angeles County, California",Estimate!!Total population,3884,Estimate,Total population
...,...,...,...,...,...,...,...
4683,2019,1400000US06037980025,"Census Tract 9800.25, Los Angeles County, Cali...",Margin of Error!!Total population,12,Margin of Error,Total population
4684,2019,1400000US06037980026,"Census Tract 9800.26, Los Angeles County, Cali...",Margin of Error!!Total population,18,Margin of Error,Total population
4685,2019,1400000US06037980028,"Census Tract 9800.28, Los Angeles County, Cali...",Margin of Error!!Total population,12,Margin of Error,Total population
4686,2019,1400000US06037980030,"Census Tract 9800.30, Los Angeles County, Cali...",Margin of Error!!Total population,12,Margin of Error,Total population


In [21]:
la_coc_totals = la_coc_total_df.groupby(['census year']).agg(
    total_population=('value', sum)).reset_index()

## Aggregate estimates

In [22]:
la_coc_est_df = la_coc_df[la_coc_df['measure'] == 'Estimate'].copy()

In [23]:
agg_la_est_df = la_coc_est_df.groupby(['census year', 'census demographic']).agg(
    Estimate=('value', sum)).reset_index()

In [24]:
agg_la_est_df.head(10)

,census year,census demographic,Estimate
0,2019,American Indian and Alaska Native,67610
1,2019,Asian,1355587
2,2019,Black or African American,745050
3,2019,Hispanic or Latino (of any race),4603241
4,2019,Native Hawaiian and Other Pacific Islander,23460
5,2019,Not Hispanic or Latino,4667500
6,2019,White,4707974
7,2019,White alone,2335886


## Calculate error for aggregation

Because I'm deriving my own estimates (summing census tracts within CA-600), I want to confirm that the demographic data I'm using is still reliable by calculating coefficients of variance for these estimates as well. For this I consulted ["Calculating Measures Of Error For Derived Estimates"](https://web.archive.org/web/20220119121038/https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_general_handbook_2020_ch08.pdf) (specifically page 60) of the American Community Survey handbook.

In [25]:
la_coc_moe_df = la_coc_df[la_coc_df['measure'] == 'Margin of Error'].copy()

In [26]:
la_coc_moe_df.rename(columns={'value': 'MOE'}, inplace=True)

The American Community Survey reports 90% confidence level margins of error (p.3, [Worked Examples for Approximating Standard Errors
Using American Community Survey Data](https://web.archive.org/web/20220120042207/https://www2.census.gov/programs-surveys/acs/tech_docs/accuracy/2020_ACS_Accuracy_Document_Worked_Examples.pdf)).

For each component estimate:

$$\mathrm{SE = \frac{MOE}{1.645}}$$

In [27]:
la_coc_moe_df['SE'] = la_coc_moe_df['MOE']/1.645

### Example

#### Estimate

For the aggregate estimate $\mathrm{\color{blue}{\hat{A}}}$:

$~~~~\mathrm{\color{blue}{\hat{A}} = \hat{A}_{1} + \hat{A}_{2} + \cdots + \hat{A}_{n}}$

For `Hispanic or Latino (of any race)` in `2010`:


In [28]:
agg_la_est_df.iloc[3:4]

,census year,census demographic,Estimate
3,2019,Hispanic or Latino (of any race),4603241


#### Error

So, for the standard error $\mathrm{\color{red}{SE}}$ of the aggregate estimate:
    
$~~~~\mathrm{\color{red}{SE}(\color{blue}{\hat{A}}) = \sqrt{\color{red}{SE}(\hat{A}_{1})]^{2} + [\color{red}{SE}(\hat{A}_{2})]^{2} + \cdots + [\color{red}{SE}(\hat{A}_{n})]^{2}}}$

For `Hispanic or Latino (of any race)` in `2010`:


| census year | demographic                      | Census Tract | SE  |
|-------------|----------------------------------|--------------|-----|
| 2010        | Hispanic or Latino (of any race) | 1            | 237 |
| 2010        | Hispanic or Latino (of any race) | 2            | 102 |
| ...         | ...                              | ...          | ... |
| 2010        | Hispanic or Latino (of any race) | 2158         | 310 |

$~~~~\mathrm{\color{red}{SE}(\color{blue}{4331506}) = \sqrt{(237)^{2} + (102)^{2} + \cdots + (310)^{2}}}$

In [29]:
def aggregate_error(values):
    squared_values = [x**2 for x in values]
    summed_sq = sum(squared_values)
    return math.sqrt(summed_sq)

In [30]:
agg_la_error_df = (
    la_coc_moe_df.groupby(['census year', 'census demographic'])
    .agg({'SE': aggregate_error}
         )
    .reset_index()
)

In [31]:
agg_la_error_df.iloc[3:4]

,census year,census demographic,SE
3,2019,Hispanic or Latino (of any race),10661.009798


$~~~~\mathrm{\color{red}{SE} = 11629}$

#### Reliability

$\mathrm{CV = \frac{\color{red}{SE}}{\color{blue}{Estimate}}}$

In [32]:
agg_la_df = pd.merge(agg_la_error_df, agg_la_est_df)

In [33]:
agg_la_df['CV'] = agg_la_df['SE']/agg_la_df['Estimate']

In [34]:
agg_la_df.iloc[3:4]

,census year,census demographic,SE,Estimate,CV
3,2019,Hispanic or Latino (of any race),10661.009798,4603241,0.002316


In this case, the coefficient of variation suggests the estimate is reliable, because the <span style="color:red">error</span> is quite small compared to the <span style="color:blue">estimate</span>:

$\mathrm{CV = \frac{\color{red}{\sim11629}}{\color{blue}{4331506}}} = 0.27\%$

Stepping back, it should be no surprise that estimates of the `Hispanic or Latino (of any race)` population in census tracts amounting to >90% of the population of Los Angeles County would be reliable.

##### Least reliable

We should check the values for which the coefficient of variation is highest:

In [35]:
agg_la_df.iloc[agg_la_df['CV'].idxmax()]

census year                                                 2019
census demographic    Native Hawaiian and Other Pacific Islander
SE                                                   1372.606551
Estimate                                                   23460
CV                                                      0.058508
Name: 4, dtype: object

Here, the CV is ~5.9%. The `Native Hawaiian and Other Pacific Islander` population estimated to have lived in the census tracts within the Los Angeles County Continuum of Care in 2010 is much smaller than the `Hispanic or Latino (of any race)` population.

Per p.16 of [Understanding and Using American Community Survey Data: What Journalists Need to Know](https://web.archive.org/web/2/https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_journalist_handbook_2020.pdf):
> [J]ournalists often wonder “How big an error margin is too big?” Statistics teachers say, “It depends.” They are right, but that is less than helpful. Consider a simpler measure [...] called the coefficient of variation (CV) [. ...] A CV of 10 percent does not seem too wobbly, but one of 50 percent probably is for most purposes.

$$\ast\ast\ast$$

In [36]:
agg_la_df.rename(columns={'Estimate': 'census estimate'},inplace=True)

In [37]:
agg_la_df

,census year,census demographic,SE,census estimate,CV
0,2019,American Indian and Alaska Native,2075.865101,67610,0.030704
1,2019,Asian,5667.295052,1355587,0.004181
2,2019,Black or African American,5305.995965,745050,0.007122
3,2019,Hispanic or Latino (of any race),10661.009798,4603241,0.002316
4,2019,Native Hawaiian and Other Pacific Islander,1372.606551,23460,0.058508
5,2019,Not Hispanic or Latino,9002.888563,4667500,0.001929
6,2019,White,11845.108800,4707974,0.002516
7,2019,White alone,6608.626223,2335886,0.002829


# All other counties

In [38]:
# def track_fields(file):
#     year = os.path.basename(file)[7:11]
#     df = pd.read_csv(file)
#     fields = [{'year': year,'position': index, 'field': key} for index, key in enumerate(df.iloc[:,0])]
#     return pd.DataFrame(fields)

In [39]:
def reshape(df):
    reshaped_df = df.melt(id_vars=['census demographic'])
    reshaped_df['geography'] = reshaped_df['variable'].str.split(
        '!!', n=1, expand=True)[0]
    reshaped_df[['geography', 'state']] = reshaped_df['geography'].str.split(
        ', ', 1, expand=True)
    reshaped_df['measure'] = reshaped_df['variable'].str.split(
        '!!', n=1, expand=True)[1]
    reshaped_df.drop(labels=['variable'], axis=1, inplace=True)
    index_cols = ['geography', 'state', 'census demographic']
    final_df = reshaped_df.pivot(
        index=index_cols, columns='measure', values='value').reset_index()
    final_df.rename_axis(None, axis=1, inplace=True)
    return final_df

In [40]:
def read_dp05(file):
    year = os.path.basename(file)[7:11]
    df = pd.read_csv(file)
    df.rename(columns={'Label (Grouping)': 'census demographic'}, inplace=True)
    # if year == '2020':
    #     ethnicity_df = pd.concat([df.iloc[74:75], df.iloc[79:80]])
    #     combination_race_df = pd.concat([df.iloc[65:66], df.iloc[80:81], df.iloc[67:72]])
    #     one_race_df = pd.concat([df.iloc[39:41],df.iloc[45:46],df.iloc[53:54], df.iloc[58:59],df.iloc[80:81]])
    # else:
    #     ethnicity_df = pd.concat([df.iloc[69:70], df.iloc[74:75]])
    #     combination_race_df = pd.concat([df.iloc[60:61],df.iloc[75:76],df.iloc[62:67]])
    #     one_race_df = pd.concat([df.iloc[34:36],df.iloc[40:41],df.iloc[48:49],df.iloc[75:76]])
    if year == '2019' or year == '2020':
        df.drop([4, 20, 21, 28, 32, 90, 91, 92, 93], axis=0, inplace=True)
    ethnicity_df = pd.concat([df.iloc[69:70], df.iloc[74:75]])
    combination_race_df = pd.concat(
        [df.iloc[75:76], df.iloc[60:67]])
    one_race_df = pd.concat(
        [df.iloc[33:36], df.iloc[40:41], df.iloc[48:49], df.iloc[75:76]])
    reshaped_ethnicity_df = reshape(ethnicity_df)
    reshaped_ethnicity_df['variable'] = 'ethnicity'
    reshaped_combination_race_df = reshape(combination_race_df)
    reshaped_combination_race_df['variable'] = 'race alone or in combination'
    reshaped_one_race_df = reshape(one_race_df)
    reshaped_one_race_df['variable'] = 'one race'
    merged_df = pd.concat(
        [reshaped_ethnicity_df, reshaped_combination_race_df, reshaped_one_race_df], ignore_index=True)
    merged_df['census demographic'] = merged_df['census demographic'].str.replace(
        '\xa0', '')
    merged_df['census year'] = year
    merged_df.rename(columns={'Estimate': 'census estimate', 'Percent': 'census percent'},inplace=True)
    # return reshaped_ethnicity_df, reshaped_combination_race_df, reshaped_one_race_df
    return merged_df

In [41]:
dp05 = pd.concat([read_dp05(file) for file in files], ignore_index=True)

NameError: name 'files' is not defined

In [ ]:
# dp05 = read_dp05('../01_inputs/USCB/DP05/ACSDP5Y2019.DP05-2022-04-01T073326.csv')

In [ ]:
dp05[dp05['variable'].isin(['ethnicity','one race'])].groupby('census demographic')[['geography','census year']].nunique()

In [ ]:
# dp05 = read_dp05_2019('../01_inputs/USCB/DP05/ACSDP5Y2019.DP05-2022-04-01T073326.csv')

In [ ]:
dp05.head(2)

In [ ]:
dp05[['census percent', 'census estimate']] = dp05[['census percent', 'census estimate']
                                     ].applymap(lambda x: re.sub('\,|\%', '', x))

In [ ]:
dp05['census percent'] = pd.to_numeric(dp05['census percent'])/100

In [ ]:
dp05.head(2)

In [ ]:
dp05.loc[dp05['census demographic'] == 'Total population', 'census percent'] = 100

In [ ]:
dp05['census estimate'] = pd.to_numeric(dp05['census estimate'])

In [ ]:
dp05_total = dp05[dp05['census demographic'] == 'Total population'].copy()

In [ ]:
dp05_total.rename(columns={'census estimate': 'total_population'}, inplace=True)

In [ ]:
dp05 = dp05[dp05['census demographic'] != 'Total population'].copy()

In [ ]:
dp05_df = pd.merge(dp05, dp05_total[['geography', 'state', 'total_population',
                                     'census year']])

In [ ]:
dp05_df[dp05_df['variable'].isin(['ethnicity','one race'])].groupby('census demographic')[['geography','census year']].nunique()

In [ ]:
pitc_county_dp05_df = dp05_df[
    (dp05_df['variable'].isin(['ethnicity', 'one race']))
    & (dp05_df['census year']=='2019')
    & (
        dp05_df['geography'].isin(
            [
                'Los Angeles County*',
                'Alameda County',
                'King County',
                'Multnomah County',
                'Sacramento County',
                'San Diego County',
            ]
        )
    )
].copy()

In [ ]:
pitc_county_dp05_df.groupby('census demographic')[['geography','census year']].nunique()

## Combine LA CoC and county data

### Percent calculation for LA CoC

In [ ]:
agg_la_df.loc[agg_la_df['census demographic'].str.contains(
    'Hispanic'), 'variable'] = 'ethnicity'

In [ ]:
agg_la_df

In [ ]:
agg_la_df['variable'].fillna('one race', inplace=True)

In [ ]:
agg_la_df

In [ ]:
la_coc_totals

In [ ]:
la_df = pd.merge(
    agg_la_df[['census year', 'census demographic', 'census estimate',  'variable']], la_coc_totals)

In [ ]:
la_df

In [ ]:
la_df['census percent'] = la_df['census estimate']/la_df['total_population']

In [ ]:
la_df

In [ ]:
la_df['geography'] = 'Los Angeles County*'

In [ ]:
la_df['state'] = 'California'

In [ ]:
la_df

In [ ]:
la_df.info()

In [ ]:
pitc_dp05_df

In [ ]:
combined_dp05_df_county = pd.concat([la_df, pitc_county_dp05_df], ignore_index=True)

In [ ]:
combined_dp05_df_county.groupby('census demographic')[['geography','census year']].nunique()

In [ ]:
combined_dp05_df_county['geography'].unique()

In [ ]:
county_fips_dict = {
    'Alameda County': '06001',
    'Los Angeles County*': '06037',
    'Sacramento County': '06067',
    'San Diego County': '06073',
    'Multnomah County': '41051',
    'King County': '53033',
}

In [ ]:
combined_dp05_df_county['county_fips'] = combined_dp05_df_county['geography'].replace(
    county_fips_dict
)

In [ ]:
combined_dp05_df_county.groupby('census demographic')[['geography','census year']].nunique()

In [ ]:
combined_dp05_df_county.to_csv(
    '../04_outputs/c01_USCB-ACS5Y-DP05-County.csv', index=False)

In [ ]:
city_dp05_df = dp05_df[(dp05_df['variable'].isin(
    ['ethnicity', 'race alone or in combination'])) & (dp05_df['geography'].str.contains('city'))].copy()

In [ ]:
geo_to_coc_dict = {
    'Los Angeles city': 'CA-600',
    'Oakland city': 'CA-502',
    'Portland city': 'OR-501',
    'Sacramento city': 'CA-503',
    'San Diego city': 'CA-601',
    'Seattle city': 'WA-500',
}

In [ ]:
city_dp05_df['coc_number'] = city_dp05_df['geography'].replace(geo_to_coc_dict)

In [ ]:
city_dp05_df['geography'] = city_dp05_df['geography'].str.replace(' city','')

In [ ]:
city_dp05_df.groupby('census demographic')[['geography','census year']].nunique()

In [ ]:
city_dp05_df.rename(
    columns={'demographic': 'census demographic'}, inplace=True)

In [ ]:
city_dp05_df.to_csv(
    '../04_outputs/c01_USCB-ACS5Y-DP05-City.csv', index=False)